<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/085_Agent_System_Components.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ✅ Agent System Components (with Planning & Reflection)

1. **Tool** – Function that does something useful (e.g., `create_plan`, `track_progress`, domain tools)
2. **Tool Registry** – Stores all tools by name for lookup by the agent
3. **ActionContext** – Holds dependencies (e.g., memory, auth, services, registry)
4. **Environment** – Executes tools using the ActionContext for dependency injection
5. **Agent** – Parses input, chooses tools, calls them via Environment
6. **Capabilities** – Modular behaviors that hook into the agent loop (e.g., **PlanFirstCapability**, **ProgressTrackingCapability**)
7. **Dependencies** – Injected values (e.g., SMTP, tokens, LLM) automatically provided to tools
8. **Wiring** – Instantiate and connect everything together (Agent + Environment + Capabilities + Tools)

---
### Notes:
- **Keep tools stateless** and narrow in scope; they should rely only on arguments and injected dependencies.
- **Use underscore `_dep` parameter names** in tool signatures to indicate injected dependencies from the ActionContext.
- **Memory** in ActionContext can store short-term state like current plans or progress logs; long-term memory strategies can be layered in later.
- **Capabilities** allow you to add or modify behavior without changing the core Agent loop.
- **Test tools individually** before integrating them to quickly catch logic or dependency issues.
- **Start simple** with a FakeModel or mock LLM to understand the loop, then connect to a real model.
- **Wiring** is your glue — keep it clear and minimal so you can swap tools, capabilities, or models easily.

---

## Notes & Tips (keep it simple)

- **Naming**: tools are verbs (`create_plan`, `track_progress`), capabilities are nouns (`PlanFirstCapability`).
- **Dependency Injection**: underscore args (e.g., `_clock`, `_smtp`) are auto‑filled from `ActionContext.deps`.
- **Stateless tools**: avoid hidden state; read/write only via `ActionContext.memory`.
- **Plan‑First habit**: always include a tiny `create_plan` tool; it stabilizes agent behavior.
- **Progress logging**: keep `track_progress` entries small (step, status, note, ts) so prints stay readable.
- **Stop conditions**: set a low `max_calls` early (5–8) to avoid runaway loops.
- **Errors**: tools should raise/return clear errors; Environment wraps them as `{ok: False, error: ...}`.
- **Pretty prints**: use `pprint` for demo outputs to see state/transcript clearly.

### Minimal success criteria for a demo run
1) A plan is created and stored at `state["current_plan"]`.
2) At least one progress entry is appended to `memory["progress"]`.
3) The agent stops with a clear final message.

### Guardrails you can add later (when ready)
- **Read‑only mode** capability that hides/blocks mutating tools.
- **Schema checks** before/after tool calls (validate required fields).
- **Time/Cost caps** in `config` (iteration/timeouts, token budgets when using a real LLM).

### Testing checklist (tiny)
- Unit test each tool as a pure function.
- Simulate DI: pass a fake dep via `deps={"clock": FakeClock}` and assert values.
- Ensure `PlanFirstCapability` is active by default; verify the first decision is `create_plan`.





## **Simple, Best-Practice Agent Recipe**

### **1. Define the Agent’s Purpose**

* One short **goal statement** — what success looks like.
* Optional constraints (time, cost, safety, privacy).
* Example: *“Help onboard new hires by sending welcome emails and scheduling meetings.”*

---

### **2. Identify Required Capabilities**

* List **domain tools** for the job.
* Add lifecycle helpers:

  * `create_plan` (PlanFirst)
  * `track_progress` (ProgressTracking)

---

### **3. List Tool Dependencies**

* For each tool, note what it needs:

  * Memory, API keys, service clients, config values.
* These become **ActionContext** entries.

---

### **4. Implement Tools**

* Always accept `action_context` as the first argument.
* Use `_dep_name` parameters for auto-injection from ActionContext.
* Keep tools **single-purpose** and stateless.

---

### **5. Build Tool Registry**

* Store tools under clear, unique names.
* Include both domain tools and planning/progress tools.

---

### **6. Assemble ActionContext**

* Put all shared dependencies in one place:

  ```python
  context = ActionContext({
      "memory": Memory(),
      "auth_token": "...",
      "smtp": SMTPService(),
      ...
  })
  ```

---

### **7. Create Environment**

* Responsible for **dependency injection**:

  * Pass `action_context` automatically.
  * Match `_dep_name` params to context keys.

---

### **8. Compose the Agent**

* Needs: Environment, ToolRegistry, Capabilities.
* Attach:

  * `PlanFirstCapability()` (optional)
  * `ProgressTrackingCapability()` (optional)

---

### **9. Wiring**

1. Register tools → ToolRegistry
2. Create ActionContext with deps
3. Create Environment with context + registry
4. Instantiate Agent with capabilities
5. Call `agent.run(goal)`

---

### **10. Test & Iterate**

* Unit test tools individually.
* Dry-run the loop and check:

  * Plan is created before actions.
  * Progress logs update.
* Add guardrails (validation, error handling) as needed.

---

✅ **Minimum Starter Pack**

* Tools: `create_plan`, `track_progress`, plus one domain tool.
* Capabilities: PlanFirst, ProgressTracking.
* Context: `memory`, plus any tool dependencies.
* Environment: DI support.


In [ ]:
"""
Ultra‑Minimal Agent Framework — Aligned to Lecture Recipe (Notebook 085.5)
Goal: Mirror the teacher's best‑practice sequence *exactly*, but keep it tiny.
No external APIs. A FakeModel simulates function calling.

Recipe order implemented below:
1) Define purpose               → GOAL string
2) Identify capabilities        → create_plan + track_progress tools; PlanFirst + ProgressTracking capabilities
3) List tool dependencies       → ActionContext holds `memory`, `config`, and any DI deps
4) Implement tools              → @register_tool, underscore DI example
5) Build tool registry          → ToolRegistry
6) Assemble ActionContext       → with deps
7) Create Environment           → executes tools + auto‑inject underscore deps
8) Compose Agent                → wires model + registry + env + capabilities
9) Wiring                       → wire_agent()
10) Test & iterate              → run_demo()
"""
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Any, Callable, Dict, List, Optional, get_type_hints
import inspect, time, pprint

# ──────────────────────────────────────────────────────────────────────────────
# 1) DEFINE PURPOSE (kept simple)
# ──────────────────────────────────────────────────────────────────────────────
GOAL = "Practice the agent loop by making a tiny plan, logging one progress update, and stopping."

# ──────────────────────────────────────────────────────────────────────────────
# Core types used throughout (simple interfaces)
# ──────────────────────────────────────────────────────────────────────────────
@dataclass
class Tool:
    name: str
    description: str
    parameters: Dict[str, Any]  # JSON‑like schema (informational)
    handler: Callable[["ActionContext", Dict[str, Any]], Any]  # (ctx, args) -> result

class ToolRegistry:
    def __init__(self):
        self._tools: Dict[str, Tool] = {}
    def register(self, tool: Tool):
        if tool.name in self._tools:
            raise ValueError(f"Tool '{tool.name}' already registered")
        self._tools[tool.name] = tool
    def get(self, name: str) -> Tool:
        return self._tools[name]
    def names(self) -> List[str]:
        return list(self._tools.keys())

@dataclass
class ActionContext:
    # 3) LIST TOOL DEPENDENCIES — everything the tools might need lives here
    memory: Dict[str, Any] = field(default_factory=dict)
    config: Dict[str, Any] = field(default_factory=dict)
    deps: Dict[str, Any] = field(default_factory=dict)  # DI bag: smtp, tokens, services, etc.
    registry: Optional[ToolRegistry] = None

# ──────────────────────────────────────────────────────────────────────────────
# 7) ENVIRONMENT — executes tools and auto‑injects underscore dependencies
# ──────────────────────────────────────────────────────────────────────────────
class Environment:
    def __init__(self, ctx: ActionContext):
        self.ctx = ctx

    def _inject_underscore_deps(self, func: Callable, args: Dict[str, Any]) -> Dict[str, Any]:
        """Fill parameters that start with '_' from ctx.deps if not provided."""
        sig = inspect.signature(func)
        filled = dict(args)
        for name, param in sig.parameters.items():
            if name.startswith('_') and name not in filled:
                key = name[1:]
                if key in self.ctx.deps:
                    filled[name] = self.ctx.deps[key]
        return filled

    def execute(self, tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
        tool = self.ctx.registry.get(tool_name)
        try:
            # For DI, peek into the original function behind the handler
            target = getattr(tool.handler, "__wrapped__", None) or tool.handler
            injected = self._inject_underscore_deps(target, arguments or {})
            result = tool.handler(self.ctx, injected)
            if not isinstance(result, dict):
                result = {"result": result}
            print(f"[env] {tool_name} <- {injected} -> {result}")
            return {"ok": True, "data": result}
        except Exception as e:
            print(f"[env] {tool_name} ERROR: {e}")
            return {"ok": False, "error": str(e)}

# ──────────────────────────────────────────────────────────────────────────────
# 5) CAPABILITIES — lifecycle modifiers (PlanFirst, ProgressTracking)
# ──────────────────────────────────────────────────────────────────────────────
class Capability:
    def on_before_loop(self, state: Dict[str, Any]):
        pass
    def on_before_model(self, state: Dict[str, Any]):
        pass
    def on_after_tool(self, state: Dict[str, Any], tool_name: str, tool_result: Dict[str, Any]):
        pass
    def on_after_model(self, state: Dict[str, Any], assistant_msg: Dict[str, Any]):
        pass

class PlanFirstCapability(Capability):
    def on_before_model(self, state: Dict[str, Any]):
        state.setdefault("guidelines", set()).add("CALL_CREATE_PLAN_FIRST")

class ProgressTrackingCapability(Capability):
    def on_after_tool(self, state: Dict[str, Any], tool_name: str, tool_result: Dict[str, Any]):
        if tool_name == "track_progress" and tool_result.get("ok"):
            state.setdefault("progress", []).append(tool_result["data"])

# ──────────────────────────────────────────────────────────────────────────────
# 6) FAKE MODEL — simulates function calling (kept trivial)
# ──────────────────────────────────────────────────────────────────────────────
class FakeModel:
    def __init__(self, require_plan_first: bool = True):
        self.require_plan_first = require_plan_first
    def respond(self, goal: str, state: Dict[str, Any], tools: List[str], transcript: List[Dict[str, Any]]):
        made_plan = "current_plan" in state
        if self.require_plan_first and ("create_plan" in tools) and not made_plan:
            return {"tool": "create_plan", "arguments": {"goal": goal}}
        if made_plan and ("track_progress" in tools) and not state.get("did_progress"):
            state["did_progress"] = True
            return {"tool": "track_progress", "arguments": {"step_index": 0, "status": "in_progress", "note": "Started."}}
        plan_len = len(state.get("current_plan", []))
        return {"final": f"Plan has {plan_len} step(s). Marked step 0 in_progress. Done."}

# ──────────────────────────────────────────────────────────────────────────────
# 8) AGENT — wires model + env + registry + capabilities
# ──────────────────────────────────────────────────────────────────────────────
class Agent:
    def __init__(self, env: Environment, registry: ToolRegistry, capabilities: Optional[List[Capability]] = None, model: Optional[FakeModel] = None, max_calls: int = 6):
        self.env = env
        self.registry = registry
        self.capabilities = capabilities or []
        self.model = model or FakeModel()
        self.max_calls = max_calls
    def run(self, goal: str) -> Dict[str, Any]:
        state: Dict[str, Any] = {}
        transcript: List[Dict[str, Any]] = []
        for cap in self.capabilities: cap.on_before_loop(state)
        for _ in range(self.max_calls):
            for cap in self.capabilities: cap.on_before_model(state)
            decision = self.model.respond(goal, state, self.registry.names(), transcript)
            if "tool" in decision:
                name, args = decision["tool"], decision.get("arguments", {})
                res = self.env.execute(name, args)
                if name == "create_plan" and res.get("ok"):
                    state["current_plan"] = res["data"].get("plan", [])
                for cap in self.capabilities: cap.on_after_tool(state, name, res)
                transcript.append({"tool": name, "args": args, "result": res})
                continue
            if "final" in decision:
                final = decision["final"]
                transcript.append({"final": final})
                for cap in self.capabilities: cap.on_after_model(state, {"content": final})
                return {"final": final, "state": state, "transcript": transcript}
        return {"final": "(stopped after max_calls)", "state": state, "transcript": transcript}

# ──────────────────────────────────────────────────────────────────────────────
# 4) IMPLEMENT TOOLS — @register_tool with underscore DI pattern
# ──────────────────────────────────────────────────────────────────────────────
registry = ToolRegistry()

def make_register_tool(registry: ToolRegistry):
    def register_tool(name: Optional[str] = None, description: str = "", parameters: Optional[Dict[str, Any]] = None):
        def deco(func):
            tool = Tool(
                name=name or func.__name__,
                description=description or (func.__doc__ or ""),
                parameters=parameters or {"type": "object", "properties": {}},
                handler=lambda ctx, args: func(ctx, **(args or {})),
            )
            tool.handler.__wrapped__ = func  # expose original for DI inspection
            registry.register(tool)
            return func
        return deco
    return register_tool

register_tool = make_register_tool(registry)

@register_tool(
    description="Create a super‑short plan of 3 steps for the given goal.",
    parameters={"type": "object", "properties": {"goal": {"type": "string"}}, "required": ["goal"]},
)
def create_plan(ctx: ActionContext, goal: str, _clock=time):
    # `_clock` shows underscore DI; Environment will inject `deps['clock']` if present.
    ts = _clock.time() if hasattr(_clock, "time") else time.time()
    plan = [
        f"Understand the goal: {goal}",
        "Choose one tiny next action",
        "Do the action and review",
    ]
    ctx.memory["last_plan_ts"] = ts
    return {"plan": plan}

@register_tool(
    description="Record a progress update for a step.",
    parameters={
        "type": "object",
        "properties": {"step_index": {"type": "integer"}, "status": {"type": "string"}, "note": {"type": "string"}},
        "required": ["step_index", "status"],
    },
)
def track_progress(ctx: ActionContext, step_index: int, status: str, note: str = ""):
    entry = {"step": int(step_index), "status": status, "note": note, "ts": time.time()}
    ctx.memory.setdefault("progress", []).append(entry)
    return {"progress_entry": entry}

# ──────────────────────────────────────────────────────────────────────────────
# 9) WIRING — build everything in the exact order of the recipe
# ──────────────────────────────────────────────────────────────────────────────

def wire_agent(require_plan_first: bool = True):
    # 6) Assemble ActionContext with deps
    ctx = ActionContext(
        memory={},
        config={"max_iterations": 6},
        deps={"clock": time},  # example DI dep used by create_plan via `_clock`
        registry=registry,
    )
    # 7) Environment
    env = Environment(ctx)
    # 8) Agent + Capabilities
    caps: List[Capability] = [ProgressTrackingCapability()]
    if require_plan_first:
        caps.insert(0, PlanFirstCapability())
    agent = Agent(env=env, registry=registry, capabilities=caps, model=FakeModel(require_plan_first=require_plan_first))
    return agent, ctx

# ──────────────────────────────────────────────────────────────────────────────
# 10) TEST & ITERATE — quick demo
# ──────────────────────────────────────────────────────────────────────────────

def run_demo():
    print("=== Demo (Lecture‑aligned) ===")
    agent, ctx = wire_agent()
    result = agent.run(goal=GOAL)
    print("Final:", result["final"])  # short concluding message
    print("State keys:", list(result["state"].keys()))
    print("Plan:", result["state"].get("current_plan"))
    print("Progress:", ctx.memory.get("progress"))
    return result

# If running as a script:
# if __name__ == "__main__":
#     run_demo()
"""
Ultra‑Minimal Agent Framework — Aligned to Lecture Recipe (Notebook 085.5)
Goal: Mirror the teacher's best‑practice sequence *exactly*, but keep it tiny.
No external APIs. A FakeModel simulates function calling.

Recipe order implemented below:
1) Define purpose               → GOAL string
2) Identify capabilities        → create_plan + track_progress tools; PlanFirst + ProgressTracking capabilities
3) List tool dependencies       → ActionContext holds `memory`, `config`, and any DI deps
4) Implement tools              → @register_tool, underscore DI example
5) Build tool registry          → ToolRegistry
6) Assemble ActionContext       → with deps
7) Create Environment           → executes tools + auto‑inject underscore deps
8) Compose Agent                → wires model + registry + env + capabilities
9) Wiring                       → wire_agent()
10) Test & iterate              → run_demo()
"""
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Any, Callable, Dict, List, Optional, get_type_hints
import inspect, time

# ──────────────────────────────────────────────────────────────────────────────
# 1) DEFINE PURPOSE (kept simple)
# ──────────────────────────────────────────────────────────────────────────────
GOAL = "Practice the agent loop by making a tiny plan, logging one progress update, and stopping."

# ──────────────────────────────────────────────────────────────────────────────
# Core types used throughout (simple interfaces)
# ──────────────────────────────────────────────────────────────────────────────
@dataclass
class Tool:
    name: str
    description: str
    parameters: Dict[str, Any]  # JSON‑like schema (informational)
    handler: Callable[["ActionContext", Dict[str, Any]], Any]  # (ctx, args) -> result

class ToolRegistry:
    def __init__(self):
        self._tools: Dict[str, Tool] = {}
    def register(self, tool: Tool):
        if tool.name in self._tools:
            raise ValueError(f"Tool '{tool.name}' already registered")
        self._tools[tool.name] = tool
    def get(self, name: str) -> Tool:
        return self._tools[name]
    def names(self) -> List[str]:
        return list(self._tools.keys())

@dataclass
class ActionContext:
    # 3) LIST TOOL DEPENDENCIES — everything the tools might need lives here
    memory: Dict[str, Any] = field(default_factory=dict)
    config: Dict[str, Any] = field(default_factory=dict)
    deps: Dict[str, Any] = field(default_factory=dict)  # DI bag: smtp, tokens, services, etc.
    registry: Optional[ToolRegistry] = None

# ──────────────────────────────────────────────────────────────────────────────
# 7) ENVIRONMENT — executes tools and auto‑injects underscore dependencies
# ──────────────────────────────────────────────────────────────────────────────
class Environment:
    def __init__(self, ctx: ActionContext):
        self.ctx = ctx

    def _inject_underscore_deps(self, func: Callable, args: Dict[str, Any]) -> Dict[str, Any]:
        """Fill parameters that start with '_' from ctx.deps if not provided."""
        sig = inspect.signature(func)
        filled = dict(args)
        for name, param in sig.parameters.items():
            if name.startswith('_') and name not in filled:
                key = name[1:]
                if key in self.ctx.deps:
                    filled[name] = self.ctx.deps[key]
        return filled

    def execute(self, tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
        tool = self.ctx.registry.get(tool_name)
        try:
            # For DI, peek into the original function behind the handler
            target = getattr(tool.handler, "__wrapped__", None) or tool.handler
            injected = self._inject_underscore_deps(target, arguments or {})
            result = tool.handler(self.ctx, injected)
            if not isinstance(result, dict):
                result = {"result": result}
            print(f"[env] {tool_name} <- {injected} -> {result}")
            return {"ok": True, "data": result}
        except Exception as e:
            print(f"[env] {tool_name} ERROR: {e}")
            return {"ok": False, "error": str(e)}

# ──────────────────────────────────────────────────────────────────────────────
# 5) CAPABILITIES — lifecycle modifiers (PlanFirst, ProgressTracking)
# ──────────────────────────────────────────────────────────────────────────────
class Capability:
    def on_before_loop(self, state: Dict[str, Any]):
        pass
    def on_before_model(self, state: Dict[str, Any]):
        pass
    def on_after_tool(self, state: Dict[str, Any], tool_name: str, tool_result: Dict[str, Any]):
        pass
    def on_after_model(self, state: Dict[str, Any], assistant_msg: Dict[str, Any]):
        pass

class PlanFirstCapability(Capability):
    def on_before_model(self, state: Dict[str, Any]):
        state.setdefault("guidelines", set()).add("CALL_CREATE_PLAN_FIRST")

class ProgressTrackingCapability(Capability):
    def on_after_tool(self, state: Dict[str, Any], tool_name: str, tool_result: Dict[str, Any]):
        if tool_name == "track_progress" and tool_result.get("ok"):
            state.setdefault("progress", []).append(tool_result["data"])

# ──────────────────────────────────────────────────────────────────────────────
# 6) FAKE MODEL — simulates function calling (kept trivial)
# ──────────────────────────────────────────────────────────────────────────────
class FakeModel:
    def __init__(self, require_plan_first: bool = True):
        self.require_plan_first = require_plan_first
    def respond(self, goal: str, state: Dict[str, Any], tools: List[str], transcript: List[Dict[str, Any]]):
        made_plan = "current_plan" in state
        if self.require_plan_first and ("create_plan" in tools) and not made_plan:
            return {"tool": "create_plan", "arguments": {"goal": goal}}
        if made_plan and ("track_progress" in tools) and not state.get("did_progress"):
            state["did_progress"] = True
            return {"tool": "track_progress", "arguments": {"step_index": 0, "status": "in_progress", "note": "Started."}}
        plan_len = len(state.get("current_plan", []))
        return {"final": f"Plan has {plan_len} step(s). Marked step 0 in_progress. Done."}

# ──────────────────────────────────────────────────────────────────────────────
# 8) AGENT — wires model + env + registry + capabilities
# ──────────────────────────────────────────────────────────────────────────────
class Agent:
    def __init__(self, env: Environment, registry: ToolRegistry, capabilities: Optional[List[Capability]] = None, model: Optional[FakeModel] = None, max_calls: int = 6):
        self.env = env
        self.registry = registry
        self.capabilities = capabilities or []
        self.model = model or FakeModel()
        self.max_calls = max_calls
    def run(self, goal: str) -> Dict[str, Any]:
        state: Dict[str, Any] = {}
        transcript: List[Dict[str, Any]] = []
        for cap in self.capabilities: cap.on_before_loop(state)
        for _ in range(self.max_calls):
            for cap in self.capabilities: cap.on_before_model(state)
            decision = self.model.respond(goal, state, self.registry.names(), transcript)
            if "tool" in decision:
                name, args = decision["tool"], decision.get("arguments", {})
                res = self.env.execute(name, args)
                if name == "create_plan" and res.get("ok"):
                    state["current_plan"] = res["data"].get("plan", [])
                for cap in self.capabilities: cap.on_after_tool(state, name, res)
                transcript.append({"tool": name, "args": args, "result": res})
                continue
            if "final" in decision:
                final = decision["final"]
                transcript.append({"final": final})
                for cap in self.capabilities: cap.on_after_model(state, {"content": final})
                return {"final": final, "state": state, "transcript": transcript}
        return {"final": "(stopped after max_calls)", "state": state, "transcript": transcript}

# ──────────────────────────────────────────────────────────────────────────────
# 4) IMPLEMENT TOOLS — @register_tool with underscore DI pattern
# ──────────────────────────────────────────────────────────────────────────────
registry = ToolRegistry()

def make_register_tool(registry: ToolRegistry):
    def register_tool(name: Optional[str] = None, description: str = "", parameters: Optional[Dict[str, Any]] = None):
        def deco(func):
            tool = Tool(
                name=name or func.__name__,
                description=description or (func.__doc__ or ""),
                parameters=parameters or {"type": "object", "properties": {}},
                handler=lambda ctx, args: func(ctx, **(args or {})),
            )
            tool.handler.__wrapped__ = func  # expose original for DI inspection
            registry.register(tool)
            return func
        return deco
    return register_tool

register_tool = make_register_tool(registry)

@register_tool(
    description="Create a super‑short plan of 3 steps for the given goal.",
    parameters={"type": "object", "properties": {"goal": {"type": "string"}}, "required": ["goal"]},
)
def create_plan(ctx: ActionContext, goal: str, _clock=time):
    # `_clock` shows underscore DI; Environment will inject `deps['clock']` if present.
    ts = _clock.time() if hasattr(_clock, "time") else time.time()
    plan = [
        f"Understand the goal: {goal}",
        "Choose one tiny next action",
        "Do the action and review",
    ]
    ctx.memory["last_plan_ts"] = ts
    return {"plan": plan}

@register_tool(
    description="Record a progress update for a step.",
    parameters={
        "type": "object",
        "properties": {"step_index": {"type": "integer"}, "status": {"type": "string"}, "note": {"type": "string"}},
        "required": ["step_index", "status"],
    },
)
def track_progress(ctx: ActionContext, step_index: int, status: str, note: str = ""):
    entry = {"step": int(step_index), "status": status, "note": note, "ts": time.time()}
    ctx.memory.setdefault("progress", []).append(entry)
    return {"progress_entry": entry}

# ──────────────────────────────────────────────────────────────────────────────
# 9) WIRING — build everything in the exact order of the recipe
# ──────────────────────────────────────────────────────────────────────────────

def wire_agent(require_plan_first: bool = True):
    # 6) Assemble ActionContext with deps
    ctx = ActionContext(
        memory={},
        config={"max_iterations": 6},
        deps={"clock": time},  # example DI dep used by create_plan via `_clock`
        registry=registry,
    )
    # 7) Environment
    env = Environment(ctx)
    # 8) Agent + Capabilities
    caps: List[Capability] = [ProgressTrackingCapability()]
    if require_plan_first:
        caps.insert(0, PlanFirstCapability())
    agent = Agent(env=env, registry=registry, capabilities=caps, model=FakeModel(require_plan_first=require_plan_first))
    return agent, ctx

# ──────────────────────────────────────────────────────────────────────────────
# 10) TEST & ITERATE — quick demo
# ──────────────────────────────────────────────────────────────────────────────

# def run_demo():
#     print("=== Demo (Lecture‑aligned) ===")
#     agent, ctx = wire_agent()
#     result = agent.run(goal=GOAL)
#     print("Final:", result["final"])  # short concluding message
#     print("State keys:", list(result["state"].keys()))
#     print("Plan:", result["state"].get("current_plan"))
#     print("Progress:", ctx.memory.get("progress"))
#     return result

def run_demo():
    print("\n=== Demo (Lecture‑aligned) ===")
    agent, ctx = wire_agent()
    result = agent.run(goal=GOAL)
    print("\nFinal Message:")
    pprint.pprint(result["final"], indent=4)
    print("\nState:")
    pprint.pprint(result["state"], indent=4)
    print("\nTranscript:")
    pprint.pprint(result["transcript"], indent=4)
    return result

# If running as a script:
if __name__ == "__main__":
    run_demo()



=== Demo (Lecture‑aligned) ===
[env] create_plan <- {'goal': 'Practice the agent loop by making a tiny plan, logging one progress update, and stopping.', '_clock': <module 'time' (built-in)>} -> {'plan': ['Understand the goal: Practice the agent loop by making a tiny plan, logging one progress update, and stopping.', 'Choose one tiny next action', 'Do the action and review']}
[env] track_progress <- {'step_index': 0, 'status': 'in_progress', 'note': 'Started.'} -> {'progress_entry': {'step': 0, 'status': 'in_progress', 'note': 'Started.', 'ts': 1754861658.087069}}

Final Message:
'Plan has 3 step(s). Marked step 0 in_progress. Done.'

State:
{   'current_plan': [   'Understand the goal: Practice the agent loop by '
                        'making a tiny plan, logging one progress update, and '
                        'stopping.',
                        'Choose one tiny next action',
                        'Do the action and review'],
    'did_progress': True,
    'guidelines': {'CAL